In [2]:
import pandas as pd
import numpy as np
import math

from keras.layers import Embedding, Dense, LSTM, Dense, Input, concatenate
from keras.models import Model
from keras.utils import plot_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [3]:
#pd.read_csv('geniune_accountsTw.csv',  encoding='utf-8', engine='python')
#pd.read_csv('social_spambots_1Tw.csv', encoding='utf-8',  engine='python')
#pd.read_csv( 'social_spambots_2Tw.csv', encoding='utf-8',  engine='python')
#pd.read_csv( 'social_spambots_3Tw.csv', encoding='utf-8',  engine='python')
#bot_accounts = pd.concat([pd.read_csv('social_spambots_1Tw.csv'), pd.read_csv('social_spambots_2Tw.csv'), pd.read_csv('social_spambots_3Tw.csv')]).reset_index(drop=True)
bot_accounts = pd.read_csv('social_spambots_1Tw.csv')
clean_accounts = pd.read_csv('geniune_accountsTw.csv')
#bot_accounts = pd.concat([pd.read_csv('social_spambots_1.csv'), pd.read_csv('social_spambots_2.csv'), pd.read_csv('social_spambots_3.csv')]).reset_index(drop=True)
#clean_accounts = pd.read_csv('geniune_accounts.csv')
requiredColumns = ['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions']
bot_accounts = bot_accounts[requiredColumns]
clean_accounts = clean_accounts[requiredColumns]
def clean_df(df):
    
    df['retweet_count'] = df['retweet_count'].apply(lambda x: 1 if x==x else 0)
    df['reply_count'] = df['reply_count'].apply(lambda x: 1 if x==x else 0)
    df['favorite_count'] = df['favorite_count'].apply(lambda x: 1 if x==x else 0)
    df['num_hashtags'] = df['num_hashtags'].apply(lambda x: 1 if x==x else 0)
    df['num_urls'] = df['num_urls'].apply(lambda x: 1 if x==x else 0)
    df['num_mentions'] = df['num_mentions'].apply(lambda x: 1 if x==x else 0)
    #df['profile_use_background_image'] = df['profile_use_background_image'].apply(lambda x: 1 if x==x else 0)
    #df['verified']=df['verified'].apply(lambda x: 1 if x==x else 0)
    #df['is_protected']=df['protected'].apply(lambda x: 1 if x==x else 0)
    #df['default_profile'] = df['default_profile'].apply(lambda x: 0 if x==x else 1)
    #df = df.rename(index=str, columns={"screen_name": "username", "statuses_count": "total_tweets", "friends_count": "total_following", "followers_count": "total_followers", "favourites_count": "total_likes"})
    return df[['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls','num_mentions']]

bot_accounts = clean_df(bot_accounts)
clean_accounts = clean_df(clean_accounts)
#print(bot_accounts)








    



C:\Users\STUDENT\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\STUDENT\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
bot_accounts['BotOrNot'] = 1
clean_accounts['BotOrNot'] = 0
#print(bot_accounts)
#print(clean_accounts)

combined_df = pd.concat([bot_accounts, clean_accounts])

new_df = combined_df.sample(frac=1).reset_index(drop=True)

In [5]:
training_df = new_df[:int(combined_df.shape[0] * 0.8)]
test_df = new_df[int(combined_df.shape[0] * 0.8):]



columns_to_standardize = ['retweet_count', 'reply_count', 'favorite_count', 'num_hashtags', 'num_urls','num_mentions']

training_df_mean = training_df[columns_to_standardize].mean()
training_df_std = training_df[columns_to_standardize].std()

#training_df[columns_to_standardize] = (training_df[columns_to_standardize] - training_df_mean)/training_df_std
#test_df[columns_to_standardize] = (test_df[columns_to_standardize] - training_df_mean)/training_df_std

# training_df_mean = training_df.mean()
# training_df_std = training_df.std()

# training_df = (training_df - training_df_mean)/training_df_std
# test_df = (test_df - training_df_mean)/training_df_std

# max_vals = training_df.max()

# training_df = training_df/max_vals
# test_df = test_df/max_vals

In [ ]:
#X_train = training_df.drop(['BotOrNot', 'is_protected'], axis=1).values
#y_train = training_df['BotOrNot'].values.reshape(-1,1)
#print(X_train)
X_train = training_df
y_train = training_df['BotOrNot'].values.reshape(-1,1)

X_test = test_df
y_test = test_df['BotOrNot'].values.reshape(-1,1)
print()
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
sm = SMOTEENN()
X_train, y_train = sm.fit_sample(X_train, y_train)
#print(X_train)
from sklearn.ensemble import AdaBoostClassifier

# Import Support Vector Classifier
from sklearn.svm import SVC
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
svc=SVC(probability=True, kernel='linear')

# Create adaboost classifer object
abc =AdaBoostClassifier(n_estimators=50, base_estimator=svc,learning_rate=1)

# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)


# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
inp = Input(shape=[11])

another = Dense(500, activation='relu')(inp)
another = Dense(200, activation='relu')(another)
another = Dense(1, activation='sigmoid')(another)

mod = Model(inp, another)
mod.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
training = mod.fit(x=X_train, y=y_train, batch_size=200, epochs=200, validation_data=(X_test, y_test))

In [67]:
mod.save('my_model.h5')